# 对原数据集进行数据抽取

In [51]:
import io
import os
import json
import jsonlines
import tokenize
import nltk
import random
from nltk.tokenize import word_tokenize # nltk.download('punkt') # 只下载一次即可
from d2l import torch as d2l

In [3]:
# 获取字符在字符串中最后出现的位置   
def getNum(target, charStr):
    
    id = target
    pos = id.find(charStr)    #此处id.find返回的参数为字符在其出现的位置
    count = 0
    while pos !=-1:
        count = count + 1
        # print('@出现在第{}个位置'.format(pos))
        if count == 4:
            return pos
        pos = (id.find(charStr,pos+1))
        
# tokenize code分词(有一个缺陷是：不可以处理不完整代码 例如括号不匹配)
def tokenize_code(origin_line):
    
    code_snippet = origin_line
    # 将代码片段转换为字节流
    code_bytes = code_snippet.encode('utf-8')
    # 使用 tokenize 模块对代码片段进行分词
    tokens = tokenize.tokenize(io.BytesIO(code_bytes).readline)
    result_line = ""
    # 输出每个 token 的类型和值
    for token in tokens:
        result_line = result_line + " " + token.string
    return result_line
    
# tokenize 自然语言分词
def tokenize_nlp(text):
    result = ""
    tokens = word_tokenize(text)
    for token in tokens:
        result = result + " " + token
    return result

In [13]:
# import nltk

# # 使用nltk自然语言tokenizer 但是针对于代码效果不好 例子：'file.data.size'解析不出
# # nltk.download('punkt') # 只下载一次即可
# from nltk.tokenize import word_tokenize

# text = """ newline_gradu KEEP  module.exports = function fileItem (props) { newline_gradu KEEP          } newline_gradu KEEP        </h4> newline_gradu KEEP        <div class="UppyDashboardItem-status"> newline_gradu DEL         ${file.data.size && html`<div class="UppyDashboardItem-statusSize">${prettyBytes(file.data.size)}</div>`} newline_gradu ADD         ${isNaN(file.data.size) ? '' : html`<div class="UppyDashboardItem-statusSize">${prettyBytes(file.data.size)}</div>`} newline_gradu KEEP          ${file.source && html`<div class="UppyDashboardItem-sourceIcon"> newline_gradu KEEP              ${acquirers.map(acquirer => { newline_gradu KEEP                if (acquirer.id === file.source) return html`<span title="${props.i18n('fileSource')}: ${acquirer.name}">${acquirer.icon()}</span>`"""
# tokens = word_tokenize(text)
# print(tokens)

## commentsGenerate 数据集处理

In [4]:
# 数据提取和special token处理等
def data_text(json_path, code_path, msg_path, codeToken_path, msgToken_path, trainPair_path):
    
    file_code = open(code_path, 'w', encoding="utf-8")
    file_msg = open(msg_path, 'w', encoding="utf-8")
    file_codeToken = open(codeToken_path, 'w', encoding="utf-8")
    file_msgToken = open(msgToken_path, 'w', encoding="utf-8")
    file_trainPair = open(trainPair_path, 'w', encoding="utf-8")
    
    with jsonlines.open(json_path) as reader:
        for obj in reader:
            if obj['msg'] != "":
                target_index = getNum(obj['patch'], '@')
                # 获取@最后一个位置使用字符串切片
                temp_line = obj['patch'][target_index+1:]
                # 数据预处理替换special token
                temp_line = temp_line.replace("\n+", " NEWLINE_NJUPT ADD_NJUPT ")
                temp_line = temp_line.replace("\n-", " NEWLINE_NJUPT DEL_NJUPT ")
                temp_line = temp_line.replace("\n", " NEWLINE_NJUPT KEEP_NJUPT ")
                temp_line = temp_line.replace("\t", " TABLE_NJUPT ")
                if not temp_line.startswith(' NEW'):
                    temp_line = " NEWLINE_NJUPT KEEP_NJUPT " + temp_line
                file_code.write(temp_line + '\n')
                
                file_msg.write(obj['msg'] + '\n')
                file_msgToken.write(tokenize_nlp(obj['msg']) + '\n')
                
                # 下面是code tokenize化操作
                result_line = ""
                code_snippet = temp_line
                code_bytes = code_snippet.encode('utf-8')
                try:
                    # 使用 tokenize 模块对代码片段进行分词
                    tokens = tokenize.tokenize(io.BytesIO(code_bytes).readline)
                    # 输出每个 token 的类型和值
                    for token in tokens:
                        result_line = result_line + " " + token.string
                except tokenize.TokenError:
                    pass
                result_line = result_line[9:]
                file_codeToken.write(result_line + '\n')
                file_trainPair.write(result_line + '\t' + tokenize_nlp(obj['msg']) + '\n')
                

        # 文件流关闭
        file_code.close()
        file_msg.close()
        file_codeToken.close()
        file_msgToken.close()
        file_trainPair.close()

In [5]:
#生成code and comment 以及tokenize文件(测试数据集)
data_text("G:\\now\\2023graduation\\authorCode\\数据集\\Comment_Generation\\msg-test.jsonl", 
          '..\\dataTry\\codeReviewer\\commentsGeneration\\test_code.txt', '..\\dataTry\\codeReviewer\\commentsGeneration\\test_msg.txt',
          '..\\dataTry\\codeReviewer\\commentsGeneration\\test_codeTokens.txt','..\\dataTry\\codeReviewer\\commentsGeneration\\test_msgTokens.txt',
          '..\\dataTry\\codeReviewer\\commentsGeneration\\test_trainPair.txt')


In [ ]:
#生成code and comment 以及tokenize文件(训练数据集)
data_text("G:\\now\\2023graduation\\authorCode\\数据集\\Comment_Generation\\msg-train.jsonl", 
          '..\\dataTry\\codeReviewer\\commentsGeneration\\train_code.txt', '..\\dataTry\\codeReviewer\\commentsGeneration\\train_msg.txt',
          '..\\dataTry\\codeReviewer\\commentsGeneration\\train_codeTokens.txt','..\\dataTry\\codeReviewer\\commentsGeneration\\train_msgTokens.txt',
          '..\\dataTry\\codeReviewer\\commentsGeneration\\train_trainPair.txt')

In [6]:
#生成code and comment 以及tokenize文件(验证数据集)
data_text("G:\\now\\2023graduation\\authorCode\\数据集\\Comment_Generation\\msg-valid.jsonl", 
          '..\\dataTry\\codeReviewer\\commentsGeneration\\valid_code.txt', '..\\dataTry\\codeReviewer\\commentsGeneration\\valid_msg.txt',
          '..\\dataTry\\codeReviewer\\commentsGeneration\\valid_codeTokens.txt','..\\dataTry\\codeReviewer\\commentsGeneration\\valid_msgTokens.txt',
          '..\\dataTry\\codeReviewer\\commentsGeneration\\valid_trainPair.txt')


## commentsDenoise 数据集处理

In [86]:
"""数据读取code and comment"""
# 从commentGeneration中得到code and comment的数据
def read_data_nmt():
    with open(os.path.join("..\\dataTry\\codeReviewer\\commentsGeneration", 'test_trainPair.txt'), 'r', encoding='utf-8') as f:
        return f.read()
# 读取code放到source里，读取comment数据放到target里
source, target = d2l.tokenize_nmt(d2l.preprocess_nmt(read_data_nmt()), None)
print(F"读取到数据的总长度为：{len(target)}")


读取到数据的总长度为：10169


In [87]:
"""mask comment 并生成数据"""
for i in range(len(target)):
    target[i] = target[i][1:len(target[i])]

valid_tag = open('..\\dataTry\\codeReviewer\\commentsDenoise\\test_tag.txt', 'w', encoding="utf-8")
valid_comments = open('..\\dataTry\\codeReviewer\\commentsDenoise\\test_comments.txt', 'w', encoding="utf-8")
valid_trainPair = open('..\\dataTry\\codeReviewer\\commentsDenoise\\test_trainPair.txt', 'w', encoding="utf-8")

for i in range(len(target)):
    tags = {}
    # 取15%个随机数
    temp_width = range(0,len(target[i]))
    random_num = random.sample(temp_width, int(len(target[i]) * 0.15))
    random_num.sort()
    length_num = len(random_num)
    # 若长度的15%为0 则默认mask第二个字符即可
    if len(random_num) == 0:
        length_num=1
        random_num.append(1)
    count = 0
    for j in range(length_num):
        tags.update({"TAG" + str(count): target[i][random_num[j]]})
        target[i][random_num[j]] = "TAG" + str(count)
        count = count + 1
    temp_comment = ""
    for n in range(len(target[i])): temp_comment = temp_comment + target[i][n] + " "
    # 数据写入
    valid_tag.write(str(tags) + '\n')
    valid_comments.write(temp_comment + '\n')
    valid_trainPair.write(temp_comment + '\t' + str(tags) + '\n')

# 数据流关闭
valid_tag.close()
valid_comments.close()
valid_trainPair.close()
